In [87]:
# 기본 패키지
import re
import time
import pickle
import pandas as pd
from tqdm import tqdm
from tqdm import trange
import warnings
warnings.filterwarnings('ignore')

# 크롤링 패키지
import requests
from selenium import webdriver

# slack으로 작업 완료 메시지 전송을 위한 사용자 정의 로컬 패키지
import myslack

In [88]:
driver = webdriver.Chrome()  
url = 'https://www.yogiyo.co.kr/mobile/?utm_source=google&utm_medium=cpc&utm_campaign=sem_bra_orignal&utm_term=%EC%9A%94%EA%B8%B0%EC%9A%94&utm_id=sem_000001&referrer=adjust_tracker%3Dgrm2aa%26adjust_google_network%3Dg%26adjust_google_placement%3D%26adjust_campaign%3Dsem_bra_web_kr_700886066_cpc%26adjust_adgroup%3D38249737818%26adjust_creative%3D%EC%9A%94%EA%B8%B0%EC%9A%94_b&gclid=EAIaIQobChMIv9DaxIL-3AIVA6yWCh0C8glmEAAYASAAEgLbF_D_BwE#/%EC%84%9C%EC%9A%B8/138223/'
driver.get(url) 

In [90]:
# 0. 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 1. 입력한 위치로 설정하기
def set_location(location):
    print(location+'으로 위치 설정 하는중...')
    driver.find_element_by_css_selector('#search > div > form > input').click()
    driver.find_element_by_css_selector('#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default > span').click()
    driver.find_element_by_css_selector('#search > div > form > input').send_keys(location)
    driver.find_element_by_css_selector('#button_search_address > button.btn.btn-default.ico-pick').click()
    time.sleep(2)
    print(location+'으로 위치 설정 완료!')
    
# 2-1. 카테고리 페이지 Element Number Dictionary 정의
food_dict = { '프랜차이즈':3, '치킨':4, '피자&양식':5, '중국집':6,
              '한식':7, '일식&돈까스':8, '족발&보쌈':9,
              '야식':10, '분식':11, '카페&디저트':12 }

# 2-2. 카테고리 페이지로 넘어가기
def go_to_category(category):
    print(category+' 카테고리 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()
    time.sleep(2)
    print(category+' 카테고리 페이지 로드 완료!')    

# 3. 카테고리(음식점 리스트) 페이지 모두 펼치기
def stretch_list_page():
    restaurant_count = int(driver.find_element_by_css_selector('#restaurant_count').text)
    scroll_count = int((restaurant_count/20))
    print('모든 음식점 리스트 불러오기 시작...')
    for _ in trange(scroll_count):
        try:
            scroll_bottom()
            time.sleep(3)
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 음식점 리스트 불러오기 완료!')
    
# 4. 해당 카테고리 음식점 리스트 리턴
def get_restaurant_list():
    restaurant_list=[]
    restaurants = driver.find_elements_by_css_selector('#content > div > div.restaurant-list > div.col-sm-6.contract')
    for restaurant in restaurants:
        restaurant_list.append(restaurant.find_element_by_css_selector('div > table > tbody > tr > td:nth-child(2) > div > div.restaurant-name.ng-binding').text)
    return list(set(restaurant_list))

# 5. 검색창에 음식점 검색하기
def search_restaurant(restaurant_name):
    try:
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a').click()
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[13]/form/div/input').send_keys(restaurant_name)    
        driver.find_element_by_xpath('//*[@id="category_search_button"]').click()
    except Exception as e:
        print('search_restaurant 에러')
    time.sleep(2)

# 6. 검색한 음식점 클릭
def go_to_restaurant():
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[2]/div').click()
    except Exception as e:
        print('go_to_restaurant 에러')
    time.sleep(2)

# 7. 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    print('리뷰 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(2)
    print('리뷰 페이지 로드 완료!')
    
# 8. 리뷰 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)
    
# 9. 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            click_more_review()
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 리뷰 불러오기 완료!')
        
# 10. 해당 음식점의 모든 리뷰 객체 리턴
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews
        
# 11. 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    print('페이지 돌아가기중...')
    driver.execute_script("window.history.go(-1)")
    time.sleep(4)
    print('페이지 돌아가기 완료!')
    
# 12. 크롤링과 결과 데이터를 pickle 파일로 저장
def save_pickle(location, category, yogiyo_df):
    pickle.dump(yogiyo_df, open('./data/{}_{}_df.pkl'.format(location, category),'wb'))
    print('{} {} pikcle save complete!'.format(location, category))

---

In [91]:
# 13. 크롤링 메인 함수
def yogiyo_crawling(location, category):
    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                                   'Total','Taste','Quantity','Delivery','Date'])
    
    try:
        set_location(location) # 검색할 지역 설정
        go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감

        print('Start [ {} - {} ] Crawling...'.format(location, category))
        
        stretch_list_page() # 카테고리(음식점 리스트) 페이지 모두 펼치기
        restaurant_list = get_restaurant_list() # 해당 카테고리 음식점 리스트 받아오기
        
        for restaurant_name in restaurant_list:
            try:
                print('********** '+restaurant_name+' - '+str(restaurant_list.index(restaurant_name)+1)
                      +'/'+str(len(restaurant_list))+' 번째 **********')
                
                search_restaurant(restaurant_name) # 음식점을 순서대로 검색
                go_to_restaurant() # 검색한 음식점 클릭             
                go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
                stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴

                for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                    try:
                        df.loc[len(df)] = { 
                            'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                            'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                            'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                            'Review':review.find_element_by_css_selector('p').text,
                            'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                            'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                            'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                            'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                            'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                        }
                    except Exception as e:
                        print('리뷰 페이지 에러')
                        print(e)
                        pass
                    
            except Exception as e:
                print('*** '+restaurant_name+' *** 음식점 페이지 에러')
                go_back_page()
                print(e)
                pass
            
            print('음식점 리스트 페이지로 돌아가는중...')
            go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
            time.sleep(2)
            
    except Exception as e:
        print('음식점 리스트 페이지 에러')
        print(e)
        pass
        
    print('End of [ {} - {} ] Crawling!'.format(location, category))
    save_pickle(location, category, df)
    myslack.send_slack('{} {} crawling finish!!!'.format(location, category))
    
    return df 

---

In [92]:
%%time 

location='역삼동'
category='족발&보쌈'

yogiyo = yogiyo_crawling(location, category)

역삼동으로 위치 설정 하는중...
역삼동으로 위치 설정 완료!
족발&보쌈 카테고리 페이지 로드중...


  0%|          | 0/3 [00:00<?, ?it/s]

족발&보쌈 카테고리 페이지 로드 완료!
Start [ 역삼동 - 족발&보쌈 ] Crawling...
모든 음식점 리스트 불러오기 시작...


100%|██████████| 3/3 [00:09<00:00,  3.01s/it]


모든 음식점 리스트 불러오기 완료!
********** 홍대칼국수와족발-강남점 - 1/60 번째 **********
리뷰 페이지 로드중...


...

리뷰 페이지 로드 완료!
모든 리뷰 불러오기 시작...


  0%|          | 0/76 [00:00<?, ?it/s]

모든 리뷰 불러오기 완료!


100%|██████████| 76/76 [00:21<00:00,  3.50it/s]


음식점 리스트 페이지로 돌아가는중...
페이지 돌아가기중...
페이지 돌아가기 완료!
End of [ 역삼동 족발&보쌈 ] Crawling!
<Response [200]>
CPU times: user 1min 30s, sys: 6.14 s, total: 1min 36s
Wall time: 2h 36min 15s


In [93]:
yogiyo.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
7157,두배족발,nh**님,,그럭저럭 맛있네요,3,,,,2014년 10월 23일
7158,두배족발,mi**님,,야식으로 시켜먹은 두배족발 정말 맛있어요 고기가 어쩜 이렇게 냄새가 한개도 안나고~...,5,,,,2014년 10월 22일
7159,두배족발,mi**님,,두배족발 저번에 갔는데 기다리다 포기하고 왔는데 주문해서 먹었는데 대박 맛있네요.....,5,,,,2014년 10월 18일
7160,두배족발,oj**님,,반반 메뉴 괜찮네요. 다만 족발집 다 주는 막국수 서비스가 없어서 좀 아쉽지만 족발...,5,,,,2014년 10월 12일
7161,두배족발,손님,,인터넷 검색해보고 평이좋아서 시켜먹었는데 진짜 맛있네요. 요긴 추천!!,5,,,,2014년 10월 11일


In [94]:
pickle.dump(yogiyo, open('./data/{}_{}_df.pkl'.format(location, category),'wb'))

In [95]:
yogiyo_df = pickle.load(open('./data/{}_{}_df.pkl'.format(location, category),'rb'))

In [96]:
yogiyo_df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
7157,두배족발,nh**님,,그럭저럭 맛있네요,3,,,,2014년 10월 23일
7158,두배족발,mi**님,,야식으로 시켜먹은 두배족발 정말 맛있어요 고기가 어쩜 이렇게 냄새가 한개도 안나고~...,5,,,,2014년 10월 22일
7159,두배족발,mi**님,,두배족발 저번에 갔는데 기다리다 포기하고 왔는데 주문해서 먹었는데 대박 맛있네요.....,5,,,,2014년 10월 18일
7160,두배족발,oj**님,,반반 메뉴 괜찮네요. 다만 족발집 다 주는 막국수 서비스가 없어서 좀 아쉽지만 족발...,5,,,,2014년 10월 12일
7161,두배족발,손님,,인터넷 검색해보고 평이좋아서 시켜먹었는데 진짜 맛있네요. 요긴 추천!!,5,,,,2014년 10월 11일


In [97]:
print(len(yogiyo_df['Restaurant'].unique()))
yogiyo_df['Restaurant'].value_counts()

55


도야족발-신사점         2010
제일쌈정식             973
선영이네닭요리           377
호세야오리바베큐-논현점      348
홍대칼국수와족발-강남점      342
돼지가바다에빠진날         275
사또통족발             226
신사동송이족발           210
마왕족발-강남점          195
이가족발-논현동          179
강남족발              178
가장맛있는족발-선정릉점      176
서초냉면              167
봉족발               150
두꺼비왕족발보쌈홍어감자탕     134
장충동왕족발보쌈-강남점      128
장수왕족발보쌈-서초강남점     111
야들족발과야들보쌈         104
원할매보쌈족발            91
이가네족발보쌈            90
두배족발               76
순정족발-신사점           70
족황상제-개포점           58
가장맛있는족발신논현점        55
음식공작소              49
완전1인용보쌈-강남점        47
전주맛집               47
대박보쌈-역삼점           41
돈통마늘보쌈-양재점         39
원할매족발보쌈            31
가장맛있는족발-선릉역점       23
장모님장충족발보쌈          20
원할매족발              19
장충동원조할매족발보쌈        14
오삼용족발              13
족삼리                12
소문난족발보쌈            12
탐라는족발-본점           10
낙지보쌈해물갈비찜-본점        9
할매맛족발맛보쌈            8
덕뽀리오리바베큐-역삼점        8
도야족발-강남역점           5
진배족발-압구정점           5
참족-본점               5
원조할머니왕족발보쌈          4
구일일돼지-강남점 

---

In [98]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기

---